In [1]:
import sys
import os

import argparse
import requests
from pprint import pprint
import numpy as np

sys.path.append("../")
import predictlol as pl
import config

api_key = config.api_key

username = 'delphinus6'
region = 'na1'
# match_index = 0

summoner_name, summoner_id, account_id = pl.get_summoner_ids(username, api_key, region)


In [2]:
matchlist = []
start_index = 0

response = requests.get("https://%s.api.riotgames.com/lol/match/v3/matchlists/by-account/%s?beginIndex=%s&api_key=%s" % (region, account_id, start_index, api_key))
matchlist_packet = response.json()

while len(matchlist_packet['matches']) > 0:
    matchlist.extend(matchlist_packet['matches'])
    start_index += 100
    response = requests.get("https://%s.api.riotgames.com/lol/match/v3/matchlists/by-account/%s?beginIndex=%s&api_key=%s" % (region, account_id, start_index, api_key))
    matchlist_packet = response.json()

In [3]:
matchlist[0]

{'platformId': 'NA1',
 'gameId': 2832713485,
 'champion': 11,
 'queue': 470,
 'season': 11,
 'timestamp': 1532495938761,
 'role': 'NONE',
 'lane': 'JUNGLE'}

In [25]:
for key in matchlist[0].keys():
    print(key)

platformId
gameId
champion
queue
season
timestamp
role
lane


In [6]:
match_data = pl.get_match_by_id(matchlist[0]['gameId'], api_key)
match_data

{'gameId': 2832713485,
 'platformId': 'NA1',
 'gameCreation': 1532495938761,
 'gameDuration': 1298,
 'queueId': 470,
 'mapId': 10,
 'seasonId': 11,
 'gameVersion': '8.14.238.9731',
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'teams': [{'teamId': 100,
   'win': 'Fail',
   'firstBlood': True,
   'firstTower': False,
   'firstInhibitor': False,
   'firstBaron': False,
   'firstDragon': False,
   'firstRiftHerald': False,
   'towerKills': 3,
   'inhibitorKills': 0,
   'baronKills': 0,
   'dragonKills': 0,
   'vilemawKills': 0,
   'riftHeraldKills': 0,
   'dominionVictoryScore': 0,
   'bans': [{'championId': 85, 'pickTurn': 1},
    {'championId': 31, 'pickTurn': 3},
    {'championId': 86, 'pickTurn': 5}]},
  {'teamId': 200,
   'win': 'Win',
   'firstBlood': False,
   'firstTower': True,
   'firstInhibitor': True,
   'firstBaron': False,
   'firstDragon': False,
   'firstRiftHerald': False,
   'towerKills': 7,
   'inhibitorKills': 2,
   'baronKills': 0,
   'dragonKills': 0,
   'vil

In [7]:
match_data['teams'][0]['win'] 

'Fail'

In [16]:
for participant in match_data['participantIdentities']:
    if participant['player']['summonerName'] == summoner_name:
        participant_id = participant['participantId']

stats = match_data['participants'][int(participant_id) - 1]['stats']
win = stats['win']
kills = stats['kills']
deaths = stats['deaths']
assists = stats['assists']

stats

{'participantId': 6,
 'win': True,
 'item0': 1419,
 'item1': 3124,
 'item2': 3144,
 'item3': 3006,
 'item4': 0,
 'item5': 0,
 'item6': 3348,
 'kills': 5,
 'deaths': 8,
 'assists': 8,
 'largestKillingSpree': 2,
 'largestMultiKill': 2,
 'killingSprees': 1,
 'longestTimeSpentLiving': 274,
 'doubleKills': 1,
 'tripleKills': 0,
 'quadraKills': 0,
 'pentaKills': 0,
 'unrealKills': 0,
 'totalDamageDealt': 128707,
 'magicDamageDealt': 3751,
 'physicalDamageDealt': 107562,
 'trueDamageDealt': 17393,
 'largestCriticalStrike': 0,
 'totalDamageDealtToChampions': 17885,
 'magicDamageDealtToChampions': 233,
 'physicalDamageDealtToChampions': 9911,
 'trueDamageDealtToChampions': 7739,
 'totalHeal': 9493,
 'totalUnitsHealed': 1,
 'damageSelfMitigated': 12128,
 'damageDealtToObjectives': 14698,
 'damageDealtToTurrets': 6644,
 'visionScore': 0,
 'timeCCingOthers': 0,
 'totalDamageTaken': 24449,
 'magicalDamageTaken': 6178,
 'physicalDamageTaken': 17135,
 'trueDamageTaken': 1136,
 'goldEarned': 10812,
 '

In [47]:
import time
start = time.time()

wins = 0
total = 0
index = 0
all_stats = []
for match in matchlist:
    match_data = pl.get_match_by_id(match['gameId'], api_key)
    for participant in match_data['participantIdentities']:
        if participant['player']['summonerName'] == summoner_name:
            participant_id = participant['participantId']
    time.sleep(2)

    stats = match_data['participants'][int(participant_id) - 1]['stats']
    for key in match.keys():
        stats[key] = match[key]
    all_stats.append(stats)
    if stats['win']:
        wins += 1
    total += 1
    
    now = time.time()
    delay = (now - start)
    h = int(np.floor(delay / 3600.))
    m = int(np.floor((delay - 3600. * h) / 60.))
    s = (delay - 3600. * h - 60. * m)
    print('Game %s, t = %02d:%02d:%2.3f' % (total, h, m, s))
    
with open('all_stats.json', 'w') as fn:
    json.dump(all_stats, fn)
    
print(wins, total, wins/total)

Game 1, t = 00:00:2.601
Game 2, t = 00:00:5.521
Game 3, t = 00:00:7.679
Game 4, t = 00:00:9.814
Game 5, t = 00:00:11.946
Game 6, t = 00:00:14.073
Game 7, t = 00:00:16.224
Game 8, t = 00:00:18.431
Game 9, t = 00:00:20.624
Game 10, t = 00:00:22.804
Game 11, t = 00:00:24.930
Game 12, t = 00:00:27.074
Game 13, t = 00:00:29.279
Game 14, t = 00:00:31.418
Game 15, t = 00:00:33.557
Game 16, t = 00:00:36.374
Game 17, t = 00:00:38.713
Game 18, t = 00:00:40.846
Game 19, t = 00:00:43.081
Game 20, t = 00:00:45.378
Game 21, t = 00:00:47.712
Game 22, t = 00:00:49.853
Game 23, t = 00:00:52.012
Game 24, t = 00:00:54.165
Game 25, t = 00:00:56.305
Game 26, t = 00:00:58.767
Game 27, t = 00:01:0.939
Game 28, t = 00:01:3.115
Game 29, t = 00:01:5.251
Game 30, t = 00:01:7.397
Game 31, t = 00:01:9.679
Game 32, t = 00:01:11.816
Game 33, t = 00:01:14.646
Game 34, t = 00:01:16.806
Game 35, t = 00:01:19.028
Game 36, t = 00:01:21.185
Game 37, t = 00:01:23.328
Game 38, t = 00:01:25.454
Game 39, t = 00:01:27.912
Game

In [28]:
a = time.time()


In [29]:
b = time.time()
b-a

3.284541130065918

In [40]:
'%.02d' % 5

'05'

In [46]:
import json

with open('matchlist.json', 'w') as fn:
    json.dump(matchlist, fn)